# Stream, Run Sketches, Save to Influx

In [2]:
!pip install mmh3

  Preparing metadata (setup.py) ... done
  Created wheel for mmh3: filename=mmh3-3.0.0-cp310-cp310-linux_x86_64.whl size=10840 sha256=a0171ddb30b22a5867a0a970553b8aa7442de706df50fa35b0e806c1a5b9c0ef
  Stored in directory: /home/jovyan/.cache/pip/wheels/00/62/4e/dc5414f9be1561daee8748ff8882d446d15e44ed2d997f2536
Successfully built mmh3


In [4]:
from pyspark.sql import SparkSession
import pandas as pd

df = pd.read_excel('./../SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'])

spark = SparkSession.builder.appName("Test").getOrCreate()
sdf = spark.createDataFrame(df) 
sdf.show(5)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+----------+----------+------+--------+-----------+-----------+----------+--------+----------+----------+--------+--------+--------+--------+-----+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+----------+----------+----------+----------+----------+------+------+------+------+--------+-----------+-----------+-------+----------+-----------+----------+--------+--------+-----------+-----------+-----------+-----------+-----+----------+----------+----------+----------+-----------+-----------+-----------+----------+------+------+------+------+--------+-----------+-----------+----------+----------+----------+----------+-------+-------+--------+--------+--------+-------+--------+-----------+-----------+-----------+
|              GMT +0|   FIT 101|   LIT 101|MV 101|P1_STATE|P101 Status|P102 Status|   AIT 201| AIT 202|   AIT 203|   FIT 201|  LS 201|  LS 202| LSL 203|LSLL 203|MV201|P2_STATE|P201 Status|P202 S

In [138]:
# from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt

# scaler = MinMaxScaler()

# col = np.array(sdf.select('FIT 401').collect())

# col = scaler.fit_transform(col).flatten()

# col = np.around(col).astype(int)

# # plt.plot(col)
# # plt.show()

In [67]:
param_w = 10
param_d = 3
seeds = np.random.randint(param_w, size = param_d)

cm = CountMinSketch(0, 10, seeds)

In [ ]:
for item in stream:
        print('1')
        # print(item)
        # for each row in the sketch
        for i in range(k):
            # hash the item
            hash_value = mmh3.hash(item)
            print(hash_value)
            # increment the corresponding cell in the sketch
            count_min_sketch[i, hash_value] += 1
    return count_min_sketch

In [3]:
import numpy as np
import mmh3

class CountMinSketch(object):
    ''' Class for a CountMinSketch data structure
    '''
    def __init__(self, width, depth, seeds):
        ''' Method to initialize the data structure
        @param width int: Width of the table
        @param depth int: Depth of the table (num of hash func)
        @param seeds list: Random seed list
        '''
        self.width = width
        self.depth = depth
        self.table = np.zeros([depth, width])  # Create empty table
        self.seed = seeds # np.random.randint(w, size = d) // create some seeds

    def increment(self, key):
        ''' Method to add a key to the CMS
        @param key str: A string to add to the CMS
        '''
        for i in range(0, self.depth):
            index = mmh3.hash(key, self.seed[i]) % self.width
            self.table[i, index] = self.table[i, index]+1

    def estimate(self, key):
        ''' Method to estimate if a key is in a CMS
        @param key str: A string to check
        '''
        min_est = self.width
        for i in range(0, self.depth):
            index = mmh3.hash(key, self.seed[i]) % self.width
            if self.table[i, index] < min_est:
                min_est = self.table[i, index]
        return min_est

    def merge(self, new_cms):
        ''' Method to combine two count min sketches
        @param new_cms CountMinSketch: Another CMS object
        '''
        return self.table + new_cms

In [92]:
from hashfactory import hash_function
from countminsketch import CountMinSketch

depth = 8
width = 2**22
hash_functions = [hash_function(i) for i in range(depth)]
sketch = CountMinSketch(depth, width, hash_functions)

ModuleNotFoundError: No module named 'hashfactory'

1
-142385009


IndexError: index -142385009 is out of bounds for axis 1 with size 1000000

## Flajolet-Martin Algorithm

In [144]:
print('Using Flajolet Martin Algorithm:')

def flajolet_martin(stream, name):
    maxnum=0
    for i in range(0,len(stream)):
        val= bin((1*stream[i] + 6) % 32)[2:]

        sum=0
        for j in range(len(val)-1,0,-1):

            if val[j]=='0':
                sum+=1
            else:
                break
        if sum>maxnum:
            maxnum=sum

    print('distict elements in ', name, ' are ', 2**maxnum)
    


Using Flajolet Martin Algorithm:


In [145]:
for k in ["FIT 401", "LIT 301"]:
    col = np.array(sdf.select(k).collect()).flatten().astype(int)
    # col = np.around(col)
    print(col)
    flajolet_martin(col, k)



[0 0 0 ... 0 0 0]
distict elements in  FIT 401  are  2
[883 883 883 ... 732 730 730]
distict elements in  LIT 301  are  16


## Frequency SpaceSaving

In [169]:
import math, heapq

class SpaceSavingCounter:
    def __init__(self, eps):
        self.k = math.ceil(1/eps)
        self.n = 0
        self.counts = dict()
        self.queue = []

    def inc(self, x):
        # increment total elements seen
        self.n += 1

        # x is being watched
        if x in self.counts:
            self.counts[x] += 1

        # x is not being watched
        else:
            # make room for x
            if self.n > self.k:
                while True:
                    count, tstamp, key = self.pop()
                    assert self.counts[key] >= count
                    if self.counts[key] == count:
                        del self.counts[key]
                        break
                    else:
                        self.push(self.counts[key], tstamp, key)
            else:
                count = 0

            # watch x
            self.counts[x] = count + 1
            self.push(count, self.n, x)


    def push(self, count, tstamp, key):
        heapq.heappush(
            self.queue,
            (count, tstamp, key)
        )

    def pop(self):
        return heapq.heappop(self.queue)
    

# seq = [5, 5, 9, 9, 1, 2, 3, 4, 5, 6, 7, 8, 5, 9, 5]
counter = SpaceSavingCounter(3)
for x in seq:
    counter.inc(x)

{5: 15}

In [171]:
for k in ["P601 Status","MV201", "P101 Status", "MV 501", "P301 Status"]:
    stream = np.array(sdf.select(k).collect()).flatten()
    # col = np.around(col)
    # print(col)
    # flajolet_martin(col, k)
    for x in stream:
        counter.inc(x)
    print(k,counter.counts)

P601 Status {1: 89991}
MV201 {2: 104987}
P101 Status {2: 119983}
MV 501 {2: 134979}
P301 Status {1: 149975}
